In [ ]:
## Author: Linda Karlsson, 2024

# import packages

import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import PurePath
import os
import Fns_refprot_normalization as fns

# define path
path = PurePath(os.getcwd())
parents = path.parents 

## This notebook creates figures from the generated linear regression file from Models_refprot_normalization.ipynb. 

#### To run this notebook, make sure to update the CSF and plasma dictionaries that defines what names should be used in the plot for each variable in the data. The keys should match the "biomarker" in linreg_res, and the values should be set to the tick names you want to have in the figures.



In [ ]:
## Change path to where you have the regression results.

os.chdir(str(path) + '') ## INSERT PATH
with open('linreg_results.pkl', 'rb') as f:
    linreg_res = pickle.load(f) ## INSERT NAME OF DATA FILE

In [ ]:
# Create dictionary of what the biomarkers are named in your data as key and what you want to name 
# them in the plots as value. Edit appropriately, see example below.

# EDIT CSF DICTIONARY 
csf_names = {'CSF_ptau217_Lilly': 'p-tau217 (Li)',
             'CSF_ptau181_Lilly': 'p-tau181 (Li)',
             'CSF_MTBRtau243_WashU': 'MTBR-tau243 (WU)',
             'CSF_ptau205_WashU': 'p-tau205 (WU)',
             'CSF_Ab42_Elecsys': 'Aβ42 (El)',
             'CSF_SNAP25_UGOT': 'SNAP-25 (GU)',
             'CSF_Neurogranin_NTK': 'neurogranin (NTK)',
             'CSF_Ab40_Elecsys': 'Aβ40 (El)',
             'CSF_ptau217_Lilly_refprot_normalized': 'p-tau217/Aβ40 (Li/El)',
             'CSF_ptau181_Lilly_refprot_normalized': 'p-tau181/Aβ40 (Li/El)',
             'CSF_MTBRtau243_WashU_refprot_normalized': 'MTBR-tau243/Aβ40 (WU/El)',
             'CSF_ptau205_WashU_refprot_normalized': 'p-tau205/Aβ40 (WU/El)',
             'CSF_Ab42_Elecsys_refprot_normalized': 'Aβ42/Aβ40 (El/El)',
             'CSF_SNAP25_UGOT_refprot_normalized': 'SNAP-25/Aβ40 (GU/El)',
             'CSF_Neurogranin_NTK_refprot_normalized': 'neurogranin/Aβ40 (NTK/El)'}

# EDIT PLASMA DICTIONARY
plasma_names = {'Plasma_ptau217_Lilly': 'p-tau217 (Li)',
                'Plasma_ptau181_Lilly': 'p-tau181 (Li)',
                'Plasma_ptau205l_WashU': 'p-tau205 (WU)',
                'Plasma_eMTBRtau243_WashU': 'eMTBR-tau243 (WU)',
                'Plasma_Ab42_WashU': 'Aβ42 (WU)',
                'Plasma_Ab40_WashU': 'Aβ40 (WU)',
                'Plasma_ptau217_Lilly_refprot_normalized': 'p-tau217/Aβ40 (Li/WU)',
                'Plasma_ptau181_Lilly_refprot_normalized': 'p-tau181/Aβ40 (Li/WU)',
                'Plasma_eMTBRtau243_WashU_refprot_normalized': 'eMTBR-tau243/Aβ40 (WU/WU)',
                'Plasma_ptau205_WashU_refprot_normalized': 'p-tau205/Aβ40 (WU/WU)',
                'Plasma_Ab42_WashU_refprot_normalized': 'Aβ42/Aβ40 (WU/WU)'
               }


# create a single dictionary with both csf and plasma names
names = {}
names.update(csf_names)
names.update(plasma_names)

## Create figure of difference between biomarker ratios and biomarkers alone

In [ ]:
## Pre-process linear regression results by updating biomarker names, setting 
#colors for each biomarker and add '*' according to signifiance level
linreg_res_diff = []
for result in linreg_res:
    linreg_res_diff.append(fns.pre_process_result_diff(result,names))

In [ ]:
### Plot results and save plot

# Set initial plot parameters and create subplots
plt.rcParams["figure.figsize"] = (15,15)
plt.rc('font', size=11)
fig, axs = plt.subplots(2,3,sharex=False,sharey=False)
plt.subplots_adjust(wspace=1.0, hspace=0.4)
props = dict(boxstyle='square', facecolor='white', alpha=1)
font_names = sorted([f.name for f in fm.fontManager.ttflist])

# Define titles and subplot names
titles = ['Tau PET Braak I-IV','Tau PET Braak V-VI', 'Amyloid-β PET',
         'Tau PET Braak I-IV','Tau PET Braak V-VI', 'Amyloid-β PET']
letters = ['a','b','c','d','e','f']
start=0.86

# Plot results in each subplot
for ax,tit,result,i,l in zip(axs.flat,titles,linreg_res_diff,range(8),letters):
    # Set y-range and ticks for CSF biomarkers
    res1 = res1.sort_values(by='R2')
    
     # make horizontal barplots
    ax.barh(res1['name'],res1['R2'],color=res1['color'])
    ax.barh(res1['name'],res1['R2']-res1['R2_difference'],color='darkgray')
    ax.grid(alpha=0.2)
    ax.set_title(tit,size=10,fontweight='semibold',)
    ax.set_xlabel('R$^2$',size=11)
    ax.set_xlim([0,0.85])
    # make errorbars
    ax.errorbar(res1['R2'],res1['name'],xerr=[res1['R2_difference']-res1['lower'],res1['upper']-res1['R2_difference']],
                color='black',lw=0.6,capsize=1.5,fmt='.',ms=2)#fmt='s'
    ax.text(-0.1, 1.05, l, transform=ax.transAxes,verticalalignment='top', size=11,fontweight='bold')
    ax.tick_params(axis='y',labelsize=9)
    
    # significance level
    for sig,i,R2_diff,R2,name in zip(res1['sig'],range(len(res1)),res1['R2_difference'],res1['R2'],res1['name']):
        if sig == '***':
            ax.text(start,i-0.2,sig,size=8,rotation='vertical')
        elif sig == '**':
            ax.text(start,i-0.1,sig,size=8,rotation='vertical')
        elif sig == '*':
            ax.text(start,i,sig,size=8,rotation='vertical')
        if R2_diff < 0:
            ax.barh(name,R2-R2_diff,color='gray',hatch='//////')
            ax.barh(name,R2,color='darkgray')
    
    
j = 28 
axs[0,1].text(0.07, 14.4, "CSF biomarkers in [cohort]", fontsize=14,fontweight='bold',fontname=font_names[j*5])
axs[1,1].text(0.026, 12.1, "Plasma biomarkers in [cohort]", fontsize=14,fontweight='bold',fontname=font_names[j*5])

fig.subplots_adjust(bottom=0.3, wspace=1)

# add legend
legend_elements = [Patch(facecolor='gray',
                         label='Unnormalized'),
                  Patch(facecolor='lightseagreen',
                         label='R$^2$ increase from Aβ40 normalization'),
                  Patch(facecolor='royalblue',
                         label='R$^2$ increase from np-tau normalization'),
                  Patch(facecolor='gray',hatch='//////',
                         label='R$^2$ reduction from normalization')]

axs[1,1].legend(handles=legend_elements,fancybox=False,ncol=5,loc='upper center', 
             bbox_to_anchor=(0.5, -0.2))


#save figure
plt.savefig('Figure1.pdf',bbox_inches='tight')

In [ ]:
### Plot results and save plot

# Set initial plot parameters and create subplots
plt.rcParams["figure.figsize"] = (12,12)
plt.rc('font', size=8)
fig, axs = plt.subplots(2,3,sharex=False,sharey=False)
plt.subplots_adjust(wspace=0.2, hspace=1.4)
props = dict(boxstyle='square', facecolor='white', alpha=1)

# Define titles and subplot names
titles = ['Tau PET Braak I-IV','Tau PET Braak V-VI', 'Amyloid-β PET',
         'Tau PET Braak I-IV','Tau PET Braak V-VI', 'Amyloid-β PET']
letters = ['a','b','c','d','e','f']

# Plot results in each subplot
for ax,tit,result,i,l in zip(axs.flat,titles,linreg_res_diff,range(8),letters):
    # Set y-range and ticks for CSF biomarkers
    if i < 3:
        ax.set_ylim(-0.17,0.42)
        start = 0.38 # height of '*'
        ax.set_yticks([-0.1,0,0.1,0.2,0.3])
    # Set y-range and ticks for plasma biomarkers
    else:
        ax.set_ylim(-0.17,0.25)
        start = 0.22 #height of '*'
        ax.set_yticks([-0.1,0,0.1,0.2])
    
    # make barplots
    ax.set_xlim(-0.8,len(result)-0.2)   
    ax.bar(result['name'],result['mean_diff'],color=result['color'])
    # make errorbars
    ax.errorbar(result['name'],result['mean_diff'],yerr=[result['mean_diff']-result['lower'],result['upper']-result['mean_diff']],color='black',lw=0.5,capsize=1.5,fmt='.',ms=2)
    # plot significance levels
    for sig,i,R2 in zip(result['sig'],range(len(result)),result['mean_diff']):
        if sig == '***':
            ax.text(i-0.3,start,sig,size=7,rotation='horizontal')
        elif sig == '**':
            ax.text(i-0.2,start,sig,size=7,rotation='horizontal')
        elif sig == '*':
            ax.text(i-0.1,start,sig,size=7,rotation='horizontal')
    
    # set general plot properties, titles and xlabels
    ax.axhline(0,color='black',lw=0.4,ls='dashed')
    ax.text(-0.1, 1.05, l, transform=ax.transAxes,verticalalignment='top', size=11,fontweight='bold')
    ax.grid(alpha=0.2)
    ax.set_title(tit,size=10,fontweight='semibold')
    ax.set_xticks(result['name'])
    ax.tick_params(axis='x', labelrotation = 90)

#set shared y-labels 
axs[0,0].set_ylabel('ΔR$^2$',size=11)
axs[1,0].set_ylabel('ΔR$^2$',size=11)

axs[0,1].text(1, 0.52, "CSF biomarkers in [cohort]", fontsize=14,fontweight='bold',fontname=font_names[j*5])
axs[1,1].text(0.48, 0.32, "Plasma biomarkers in [cohort]", fontsize=14,fontweight='bold',fontname=font_names[j*5])

# add legend
fig.subplots_adjust(bottom=0.3, wspace=0.2,hspace=1.2)
legend_elements = [
                  Patch(facecolor='lightseagreen',
                         label='R$^2$ change from Aβ40 normalization'),
                  Patch(facecolor='royalblue',
                         label='R$^2$ change from np-tau normalization')]

axs[1,1].legend(handles=legend_elements,fancybox=False,ncol=5,loc='upper center', 
             bbox_to_anchor=(0.5, -0.9),fontsize=9)


#save figure
plt.savefig('Figure2.pdf',bbox_inches='tight')

In [ ]:
## Pre-process linear regression results by updating biomarker names and setting 
#colors for each biomarker
for result in h2h_res:
    for bm,i in zip(result['biomarker'],range(len(result['biomarker']))):
        result.loc[i,'name'] = names[bm]
    result['color'] = fns.give_color(result)

In [ ]:
### Plot results and save plot

# Set initial plot parameters and create subplots
plt.rcParams["figure.figsize"] = (12,10.5)
plt.rc('font', size=8)
fig, axs = plt.subplots(2,3,sharex=False,sharey=False)
plt.subplots_adjust(wspace=1.1, hspace=0.34)
props = dict(boxstyle='square', facecolor='white', alpha=1)

# Define titles and subplot names
titles = ['Tau PET Braak I-IV','Tau PET Braak V-VI', 'Amyloid-β PET',
         'Tau PET Braak I-IV','Tau PET Braak V-VI', 'Amyloid-β PET']
letters = ['a','b','c','d','e','f']

# Plot results in each subplot
for ax,result,tit,l in zip(axs.flat,h2h_res,titles,letters):
    
    # sort values according to R2-score
    result = result.sort_values(by='R2_mean')
    # make horizontal barplots
    ax.barh(result['name'],result['R2_mean'],color=result['color'])
    # make errorbars
    ax.errorbar(result['R2_mean'],result['name'],xerr=[result['R2_mean']-result['R2_lower'],result['R2_upper']-result['R2_mean']],color='black',lw=0.5,capsize=1.5,fmt='.',ms=2)
    
    # set general plot properties, titles and xlabels
    ax.grid(alpha=0.2)
    ax.set_title(tit,size=10,fontweight='semibold')
    ax.set_xlabel('R-squared',size=10)
    ax.set_xlim([0,0.85])
    ax.text(-0.1, 1.05, l, transform=ax.transAxes,verticalalignment='top', size=10,fontweight='bold')
    
#save figure
plt.savefig('head_to_head_barplots.jpg',bbox_inches='tight',dpi=300)